In [1]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
lifelines 0.30.0
sklearn 1.5.2
lightgbm
xgboost 2.1.2
catboost 1.2.5


In [2]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.impute import SimpleImputer
from lifelines import NelsonAalenFitter

data_path = 'data'
model_path = 'model'

def score(df, prds):
    return df.groupby('race_group', observed=True).apply(
        lambda x: concordance_index(x['efs_time'], -prds.loc[x.index], x['efs']), include_groups=False
    ).pipe(
        lambda x: float(x.mean() - x.std(ddof=0))
    )
p2 =joblib.load(os.path.join(model_path, 'p2.joblib'))
df_train = p2.transform([os.path.join(data_path, 'train.csv')])

In [33]:
df_kmf_cv = pd.concat([
    joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb7', 'xgb7', 'cb7']
], axis=1)
df_kmf_cv.head()

,lgb7,xgb7,cb7
ID,,,
0,0.493604,0.496470,0.498945
1,0.621568,0.629487,0.648487
2,0.447219,0.452650,0.453923
3,0.670165,0.659071,0.647647
4,0.637481,0.645079,0.654801


In [24]:
df_naf_cv = pd.concat([
    joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb6', 'xgb6', 'cb6']
], axis=1)
df_naf_cv.head()

,lgb6,xgb6,cb6
ID,,,
0,-0.727016,-0.710788,-0.701036
1,-0.474285,-0.507268,-0.483968
2,-0.806291,-0.801419,-0.774680
3,-0.452092,-0.460558,-0.476675
4,-0.485533,-0.487864,-0.469562


In [25]:
df_stk = pd.concat([
    df_kmf_cv.dot([0.4, 0.3, 0.3]).rename('kmf').rank(),
    df_naf_cv.dot([0.4, 0.3, 0.3]).rename('naf').rank()
], axis=1)

In [26]:
score(df_train, df_stk.dot([0.0, 1.0]))

0.6764655470523381

In [7]:
df_cv3 = pd.concat([
    pd.concat([
        joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb3_1', 'lgb3_2', 'lgb3_3', 'lgb3_4']
    ], axis=1).dot([0.2, 0.2, 0.3, 0.3]),
    pd.concat([
        joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['xgb3_1', 'xgb3_2', 'xgb3_3', 'xgb3_4']
    ], axis=1).dot([0.2, 0.2, 0.3, 0.3]),
], axis=1)

In [21]:
df_cv4 = pd.concat([
    pd.concat([
        joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb5_1', 'lgb5_2', 'lgb5_3', 'lgb5_4']
    ], axis=1).dot([0.2, 0.2, 0.3, 0.3]),
    pd.concat([
        joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['xgb5_1', 'xgb5_2', 'xgb5_3', 'xgb5_4']
    ], axis=1).dot([0.2, 0.2, 0.3, 0.3]),
], axis=1)

In [34]:
df_stk = pd.concat([
    df_kmf_cv.dot([0.5, 0.1, 0.4]).rename('kmf').rank(),
    df_naf_cv.dot([0.5, 0.1, 0.4]).rename('naf').rank(),
    df_cv3.dot([0.8, 0.2]).rename('3').rank(),
    df_cv4.dot([0.8, 0.2]).rename('4').rank()
], axis=1)

In [38]:
score(df_train, df_stk.dot([0.00, 0.1, 0.0, 0.9]))

0.6819552175820196